<a href="https://colab.research.google.com/github/Anhbt95/geemap/blob/main/3_using%20earth%20engine%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INFO

Using earth engine data
https://book.geemap.org/chapters/03_gee_data.html

# SETUP

In [ ]:
# install packages
%pip install geemap
%pip install pygis

In [2]:
# load pacakges
import ee
import geemap

In [3]:
# authorize gee
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=2ca2MCPjMLYXoGKaBmOO4XUrCUOmGTibVErACzSFBTc&tc=zNUvDQLz1qcHwtZdHWDeFKITPbmJa66oqYhD5uwQr20&cc=gFAoDgVyoOp-sK6o24ci-VpVFIm7YAvR7pMBydYP09c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUc8IosbqMGMiVG3tDKbtVHJ-I1rWsfhXljejJge8wrXX5mHsdVA9o

Successfully saved authorization token.


# earth engine data types

## image

In [ ]:
# earth engine image
## get image
image = ee.Image('USGS/SRTMGL1_003')
image
image.getInfo()

In [ ]:
Map = geemap.Map(center=[21.79, 70.87], zoom=3)
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM')
Map

Map(center=[21.79, 70.87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

## collection

In [ ]:
# load collection
collection = ee.ImageCollection('COPERNICUS/S2_SR')
collection.limit(5)

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [ ]:
# visualize collection
Map = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR') #copernicus collection
image = collection.median() # calculate median of the collection

# setup visualization
vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(104, 10.2, 10)
Map.addLayer(image, vis, 'Sentinel-2')

#Map

In [ ]:
## filter image collection
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01') #image in 2021
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) #could pixel percentage: less than 5%
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(104, 10.2, 10)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[10.2, 104], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

## feature

In [ ]:
# create polygon with spatial info
polygon = ee.Geometry.Polygon(
    [[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]], None, False
)

# add attribute info
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})
polyFeature

# add feature to map
#Map = geemap.Map()
#Map.addLayer(polyFeature, {}, 'feature')
#Map



In [ ]:

# set feature properties
feature = (
    ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))
    .set('genus', 'Sequoia')
    .set('species', 'sempervirens')
)
newDict = {'genus': 'Brachyramphus', 'presence': 1, 'species': 'marmoratus'}
feature = feature.set(newDict)
feature


In [ ]:
# get feature properties
prop = feature.get('species')
prop

props = feature.toDictionary()
props

## feature collection

In [ ]:
# load feature collection
Map = geemap.Map()
fc = ee.FeatureCollection('TIGER/2016/Roads')
Map.setCenter(-73.9596, 40.7688, 12)
Map.addLayer(fc, {}, 'Census roads')
#Map

In [ ]:
# filter feature collection
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')

# see ee as dataframe
geemap.ee_to_df(states)

feat = states.filter(ee.Filter.eq('NAME', 'Texas'))
Map.addLayer(feat, {}, 'Texas')
Map.centerObject(feat)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
texas = feat.first()
texas.toDictionary()

In [ ]:
# filter feature by polygon
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)
Map.addLayer(fc, {}, 'Southeastern U.S.')
Map.addLayer(region, {}, "AOI")
Map.centerObject(fc)
Map


Map(bottom=3666.0, center=[30.543338954230222, -83.51806640625001], controls=(WidgetControl(options=['position…

### visualize feature collections


In [ ]:
# us states
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
#Paints the geometries of a collection onto an image
ee.Image().paint(states, )
image = ee.Image().paint(states, 0, 3)
Map.addLayer(image, {'palette': 'red'}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000080'}
Map.addLayer(states.style(**style), {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### style by attribute

In [ ]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')
Map

Map(center=[28.00142, -81.7424], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [ ]:
# setup type, colors, and fill color
types = [
    "Freshwater Forested/Shrub Wetland",
    "Freshwater Emergent Wetland",
    "Freshwater Pond",
    "Estuarine and Marine Wetland",
    "Riverine",
    "Lake",
    "Estuarine and Marine Deepwater",
    "Other",
]

colors = [
    "#008837",
    "#7FC31C",
    "#688CC0",
    "#66C2A5",
    "#0190BF",
    "#13007C",
    "#007C88",
    "#B28653",
]

#fillColor = [c + "A8" for c in colors]
fillColor = [c for c in colors]


In [ ]:
# get feature collection
fc = ee.FeatureCollection("projects/sat-io/open-datasets/NWI/wetlands/FL_Wetlands")

# check feature collection
fc_first = fc.first()
fc_first.toDictionary()

#geemap.ee_to_df(fc) #timeout error

styled_fc = geemap.ee_vector_style(
     fc, column='WETLAND_TY',
     labels=types,
     fillColor=fillColor, color='00000000'
)

In [ ]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')

Map.addLayer(styled_fc, {}, 'NWI')
Map.add_legend(title='Wetland Type', labels=types, colors=colors)
Map

## earth engine catalog

In [5]:
Map = geemap.Map()
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
from geemap.datasets import DATA

In [7]:
Map = geemap.Map(center=[40, -100], zoom=4)
dataset = ee.Image(DATA.USGS_GAP_CONUS_2011)
Map.addLayer(dataset, {}, 'GAP CONUS')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [8]:
from geemap.datasets import get_metadata

get_metadata(DATA.USGS_GAP_CONUS_2011)

HTML(value='\n            <html>\n            <body>\n                <h3>USGS GAP CONUS 2011</h3>\n          …

## getting image metadata

In [9]:
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')

In [10]:
# image band names
image.bandNames()

In [11]:
# image projection
image.select('SR_B1').projection()

In [ ]:
image.select('SR_B1').projection().nominalScale()
image.propertyNames() #property
# check certain property
image.get('CLOUD_COVER')
image.get('DATE_ACQUIRED')
image.get('system:time_start')
date = ee.Date(image.get('system:time_start'))
date.format('YYYY-MM-dd')

# image to dict
image.toDictionary()

props = geemap.image_props(image)
props

## calculating descriptive statistics

In [21]:
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')
geemap.image_min_value(image)
geemap.image_stats(image) #cal al stats

## using inspector tool

In [ ]:
# get map
Map = geemap.Map(center = (40, -100), zoom = 4)

# get dem NASA SRTM Digital Elevation 30m
dem = ee.Image(args='USGS/SRTMGL1_003')
## get property
props = geemap.image_props(image)
props.getInfo()

# get landsat
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])


In [94]:
# feature states
states = ee.FeatureCollection(args = "TIGER/2018/States")

# df ee_to_pandas
state_df = geemap.ee_to_pandas(states)
state_df

## filter
state_boundary = states.filter(ee.Filter.eq('NAME', 'California'))

In [95]:
dem_cropped = dem.clip(clip_geometry = state_boundary.geometry())

In [96]:
Map = geemap.Map(center = (40, -100), zoom = 4)
Map.addLayer(ee_object = dem_cropped, vis_params = vis_params, name = "dem_cropped")
Map



Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [97]:
# plot
# vis param
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ["#f1eef6", "#d7b5d8", "#df65b0", "#dd1c77", "#980043"]
}

# add layer to map
Map.addLayer(ee_object = dem, vis_params = vis_params, name = "dem")
Map.addLayer(ee_object = dem_cropped, vis_params = vis_params, name = "dem_cropped")
Map.addLayer(ee_object = landsat7,
             vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
             name = "landsat7")
Map.addLayer(ee_object = states,
             vis_params = {},
             name = "US states")

Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [98]:
# Filter the feature collection to get a specific state, e.g., California
state_boundary = states.filter(ee.Filter.eq('STUSPS', 'CA'))

# Get the geometry of the selected state
state_geometry = state_boundary.geometry()

# Clip the SRTM image using the state boundary
cropped_srtm = dem.clip(state_geometry)

# Add layers to the map
Map.centerObject(state_geometry, 6)
Map.addLayer(cropped_srtm, {'min': 0, 'max': 4000}, 'Cropped SRTM')
Map.addLayer(state_boundary, {'color': 'red'}, 'State Boundary')

# Display the map
Map
#Replace 'CA' with the state abbreviation of your choice if you want to crop the image using a different state. This script will display the SRTM image cropped to the boundary of the selected state using the geemap library.







Map(bottom=1851.0, center=[37.152434104671315, -119.52655155150876], controls=(WidgetControl(options=['positio…